# H07 Auto Toolchain Helpers

In [ ]:
import json

In [83]:
#todo for benchmark looo
#and within benchmakr function
widgets.IntProgress(
    value=73,
    min=0,
    max=100,
    step=1,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)

IntProgress(value=73, description='Loading:')

In [17]:
# widgets - loop variables
tfl_model_selections = widgets.SelectMultiple(
    options=sorted(glob.glob(f"TFLite-model/*full.tflite")),
    description='Select model:',
    layout=Layout(width='100%', height='200px')
)

cmsis_selections = widgets.SelectMultiple(
    options=[('none', './TFLu_benchmark-model_mbed'), ('cmsis-nn', './TFLu_benchmark-model_mbed_cmsis-nn')],
    description='Select cmsis modes:',
)


fpu_selections = widgets.SelectMultiple(
    options=[('FPU disabled',0),('FPU enabled',1)],
    description='Select FPU modes:',
)

compiler_selections = widgets.SelectMultiple(
    options=['-Ofast', '-Os'],
    description='Select compiler options:',
)



In [25]:
# compiler widgets

inferences_slider = widgets.FloatLogSlider(
    value=1,
    base=10,
    min=0, # max exponent of base
    max=4, # min exponent of base
    step=1, # exponent step
    description='No of repetition per inference'
)

target_selections = widgets.SelectMultiple(
    options=['auto', 'NUCLEO_L496ZG', 'NUCLEO_F767ZI', '...'],
    description='Select targets:',
)

arena_size_slider = widgets.IntSlider(
    value=50,
    min=0,
    max=500, 
    step=50,
    description='Arena Size [KiB]'
)

cycles_selection = widgets.Checkbox(
    value=False,
    description='Benchmark in cycles (instead of us)',
    indent=False
)
layers_selection = widgets.Checkbox(
    value=False,
    description='Benchmark with layer granularity (instead of a whole inference)',
    indent=False
)
reporting_selection = widgets.Checkbox(
    value=True,
    description='Report the results of the inference via UART',
    indent=False
)
input_selection = widgets.Checkbox(
    value=True,
    description='Enabling custom input via UART (required for automated verification)',
    indent=False
)
energy_selection = widgets.Checkbox(
    value=False,
    description='Enable custom settings for an energy measurement',
    indent=False
)

baudrate_slider = widgets.FloatLogSlider(
    value=10e6,
    base=10,
    min=4, # max exponent of base
    max=6, # min exponent of base
    step=1, # exponent step
    description='Baudrate'
)

In [22]:



columns = ['time','MCU',
           'model', 'mbed-dir', 'cmsis-nn', 'arena_size',
           'compiler_optimization','FPU_status', 'FLOPs', 'FLOPs_profiler',
           'pruned', 'weights', 'activations', 'model_type',
           'model_size', 'model_size_reduction', 
           'model_size_gzip', 'model_size_reduction_gzip', 'binary_size',
           'input_details_dtype', 'input_details_shape', 
           'output_details_dtype', 'output_details_shape',
           'tfl_interpreter_accuracy', 'tfl_interpreter_loss_crossentropy', 'tfl_interpreter_loss_meansquared',
           'inferences_per_cycle', 'tflu_mcu_benchmark_single',
           'tflu_mcu_benchmark_mean','tflu_mcu_benchmark_std', 
           'tflu_mcu_accuracy', 'tflu_mcu_loss_crossentropy', 'tflu_mcu_loss_meansquared'
            'layer',
           'layer_latency_mean',
           'layer_latency_std',
          ]

table_energy = pd.DataFrame()
table_firmware = pd.DataFrame()

In [106]:
def get_cmsis(mbed_dir):
    if 'cmsis' in mbed_dir:
        return 'cmsis-nn'
    else:
        return 'none'

In [43]:
def read_model_information(filename, series):
    pass

*todo*
- currently we only support benchmarking with layer granularity
- save all the information


In [46]:
def run_benchmark(RL, target_device, serial_port, baudrate, tfl_model_file, mbed_dir, compiler_flag, fpu_status, 
                 no_samples = 50):


    tfl_model_string = get_tfl_model_string(tfl_model_file)
    print("Benchmarking:", tfl_model_string)

    model_information = pd.Series(dtype=np.float32)
    #model_information = {}
    

    model_information['time'] = str(datetime.now())
    model_information['MCU'] = target_device
    model_information['model'] = tfl_model_string

    model_information['mbed-dir'] = mbed_dir
    model_information['cmsis-nn'] = get_cmsis(mbed_dir)

    read_model_information(tfl_model_string, model_information)

    if "Q-mixed" in tfl_model_string:
        model_information['weights'] = 'int8'
        model_information['activations'] = 'float32'
    elif "Q-full" in tfl_model_string:
        model_information['weights'] = 'int8'
        model_information['activations'] = 'int8'
    else:
        model_information['weights'] = 'float32'
        model_information['activations'] = 'float32'

    model_information['model_type'] = 'W-' + model_information['weights'] + '_A-' +model_information['activations']

    
    
    keras_model = tfl_model_string[:tfl_model_string.find('_Q')]
    # keras-flops library
    model_information['FLOPs'] = keras_flops_get_flops(f'./keras-model/{keras_model}.h5')
    # TF Profiler
    model_information['FLOPs_profiler'] = profiler_get_flops(f'./keras-model/{keras_model}.h5')

  #  input_details, output_details = get_tfl_details(tfl_model_file)

    # model_information['input_details'] = input_details
    #model_information['input_details_dtype'] = str(input_details[0]['dtype'])
 #   model_information['input_details_shape'] = str(input_details[0]['shape'])

    # model_information['output_details'] = output_details
    #model_information['output_details_dtype'] = str(output_details[0]['dtype'])
 #   model_information['output_details_shape'] = str(output_details[0]['shape'])

    # filesize of the model
    model_size, reduction = get_tfl_size(tfl_model_file, unquantized_model_file=basic_model_file)
    model_information['model_size'] = model_size
    model_information['model_size_reduction'] = reduction

    model_size, reduction = get_tfl_size(tfl_model_file, unquantized_model_file=basic_model_file, gzip=True)
    model_information['model_size_gzip'] = model_size
    model_information['model_size_reduction_gzip'] = reduction
    
    
    

    # accuracy and error
    # todo: this might need adjustments when the input and output datatype differs (e.g. when it's quantized)

    ## prediction

    # only calculate the predictions once for each model - this might take a while
#     if tfl_model_file in accuracies:
#         # it already exists
#         print("\tPredictions from local evaluation already exist. Reusing them.")
#         model_information['tfl_interpreter_accuracy'] = accuracies[tfl_model_file]
#     else:
#         # we have to do the math
#         print("\tRunning local evaluation of model using TF Lite Interpreter ...")
#         tfl_model_predictions = predict(tfl_model_file, x=x_test_normalized)
#         accuracies[tfl_model_file] = accuracy(tfl_model_predictions, y=y_test)
#         model_information['tfl_interpreter_accuracy'] = accuracies[tfl_model_file]

#         print("\tEvaluation finished.")
# 
# 
#     ## loss
#     model_information['tfl_interpreter_loss_crossentropy'] = loss_fn_crossentropy(y_test, tfl_model_predictions).numpy()
#     model_information['tfl_interpreter_loss_meansquared'] = loss_fn_meansquared(y_test, tfl_model_predictions).numpy()


    
    
    
    
    tfl_model_to_file(tfl_model_file, mbed_dir)

    write_constants(tfl_model_string, 1, image_no, mbed_dir)

    set_compiler_flag(mbed_dir, compiler_flag)
    model_information['compiler_optimization'] = compiler_flag

    toggle_fpu(mbed_dir, fpu_status)
    model_information['FPU_status'] = fpu_status

    print("\tBuilding & flashing ...")
    !cd {mbed_dir} && mbed compile -m {target_device} -t GCC_ARM --profile release --flash {arguments} > build_log.txt

    if _exit_code != 0:
        print("\tThere was an error during compilation. Skipping ...")
        model_information['error'] = 'error during compilation'
        return model_information, model_information

    print("\tFinished building & flashing.")
    
    print("\tGetting size of the binary blob ...")
    binary_filename = glob.glob(f"{mbed_dir}/BUILD/{target_device}/GCC_ARM-RELEASE/*.bin")
    model_information['binary_size'] = os.path.getsize(binary_filename[0])
    
    
    
    ## start rocketlogger 
    

    
    
    
    ## TODO: energy meausrmenet, and non layer measurement

    time.sleep(1)

    port = set_port(serial_port, baudrate)
    
    ## complete inference
#     print("\tVerifying ...")
#         mcu_predictions, benchmark_results = verify_accuracy_on_mcu(port, test=not(run_verification_on_mcu))

#     if run_verification_on_mcu:
#         model_information['tflu_mcu_benchmark_mean'] = benchmark_results.mean()
#         model_information['tflu_mcu_benchmark_std'] = benchmark_results.std()

#         model_information['tflu_mcu_accuracy'] = accuracy(mcu_predictions)

#         model_information['tflu_mcu_loss_crossentropy'] = loss_fn_crossentropy(y_test, mcu_predictions).numpy()
#         model_information['tflu_mcu_loss_meansquared'] = loss_fn_meansquared(y_test, mcu_predictions).numpy()

#     else:
#         model_information['tflu_mcu_accuracy_50'] = accuracy(mcu_predictions)                    
#         model_information['tflu_mcu_benchmark_mean_50'] = benchmark_results.mean()
#         model_information['tflu_mcu_benchmark_std_50'] = benchmark_results.std()

    
    ## Layers
    
    print("\tFirst emasurmeent which will be discarded - warm up.")
    # discard first result
    send_image(port, x_test_normalized[image_no])
    read_results(port)
    
      #  as comment for energy measurement
    json_dump = model_information.filter(items=['time', 'MCU', 'model', 'model_type', 
                                                'cmsis-nn', 'compiler_optimization',
                                                'FPU_status']).to_json()
    
    # json_dump = json.dumps(model_information)
    # start energy measurement
    
    RL.filename = f'{target_device}_{model_name}_{date.today()}'
    #RL.comment = str(json_dump)
    #print(json_dump)
    
    RL.set_config()
    
    RL.start_measurement()
    
    sample_images = x_test_normalized[image_no:image_no+no_samples]

    print(f"\tBenchmarking layers (sample of {len(sample_images)} images) ...")

    
    table_layers = pd.DataFrame()
    for sample_image in sample_images:
        send_image(port, sample_image)
        _, benchmark_result = read_results(port)
        table_layers = pd.concat([table_layers, pd.DataFrame.from_dict(benchmark_result, orient='index')], axis=1)

    port.close()
    
    RL.stop_measurement()
    
    RL.download_file()
    
    # process RL file
    filename = "./results/" + RL.filename + ".rld" 
    #df = pd.DataFrame()
    #df_layers = pd.DataFrame()
    energy_inference_table, energy_inference_layer_table = energy_analysis(filename, model_information)
    #df = pd.concat([df, energy_inference_table])
    #df_layers = pd.concat([df_layers, energy_inference_layer_table])
    

    #df.to_excel(f'results/E_{target_device}_{tfl_model_string}_aggregated_energy_inference_table_{date.today()}.xlsx')
    #df.to_pickle(f'results/E_{target_device}_{tfl_model_string}_aggregated_energy_inference_table_{date.today()}.pkl')
    energy_inference_layer_table.to_excel(f'results/E_{target_device}_{tfl_model_string}_aggregated_energy_inference_layer_table_{date.today()}.xlsx')
    energy_inference_layer_table.to_pickle(f'results/E_{target_device}_{tfl_model_string}_aggregated_energy_inference_layer_table_{date.today()}.pkl')
    

    table_layers.to_pickle(f'results/layers_raw/F_{target_device}_{tfl_model_string}_layer_raw-results_{date.today()}.pkl')
    table_layers_summary = pd.DataFrame(columns=columns)

    table_layers_summary['layer'] = table_layers.index
    table_layers_summary['layer_latency_mean'] = table_layers.mean(axis=1).values
    table_layers_summary['layer_latency_std'] = table_layers.std(axis=1).values
    #table_layers_summary.loc['sum_latency'] = table_layers_summary.sum()
    table_layers_summary.loc[:, 'layer_latency_relativ'] = table_layers_summary.loc[:]['layer_latency_mean'] / table_layers_summary.loc[:]['layer_latency_mean'].sum()    

    for key, value in model_information.iteritems():
        # print(f"key: {key}; value: {value}")
        table_layers_summary[key] = value

    ## end layers
    
    #table_layers_summary.to_pickle(f'results_layers/layers_{mbed_dir}_{tfl_model_string}.pkl')
    

    
    return table_layers_summary, energy_inference_table